In [31]:
import numpy as np
import csv
import pandas as pd
import re
import os
import json
import math

### 处理文件路径
root_path = 'assets/1npccutin/'

In [21]:
def npc_from_txt(filepath):
    f = open(filepath, 'r')
    fframe = f.readlines()
    
    #append timestamp
    df_ts = pd.DataFrame(columns=['npc_ts'])
    #append npc
    df_lc = pd.DataFrame(columns=['npc_x','npc_y','theta'])
    
    for row in fframe:
        if row.find('timestamp_sec') != -1:
            ts = re.findall("(?<=[AZaz])?[0-9.+-]+",row) #exclude "timestamp_sec:"
            ts = float(ts[0]) #convert to data format
            l_no = fframe.index(row)
            
            lc_x = re.findall("(?<=[AZaz])?[0-9.+-]+",fframe[l_no + 11])
            lc_x = float(lc_x[0])
            lc_y = re.findall("(?<=[AZaz])?[0-9.+-]+",fframe[l_no + 12])
            lc_y = float(lc_y[0])
            theta = re.findall("(?<=[AZaz])?[0-9.+-]+",fframe[l_no +15])
            theta = float(theta[0])
            
            d_0 = {'npc_ts':[ts]}
            d_df0 = pd.DataFrame(data=d_0)
            df_ts = pd.concat([df_ts, d_df0])

            d_1 = {'npc_x':[lc_x], 'npc_y':[lc_y],'theta':[theta]}
            d_df1 = pd.DataFrame(data=d_1)
            df_lc = pd.concat([df_lc, d_df1])
    df = pd.concat([df_ts, df_lc], axis=1)

    return df

In [22]:
import pandas as pd
import re

def npc_from_txt_wid(filepath):
    with open(filepath, 'r') as f:
        content = f.read()
    
    blocks = re.split(r'module_name: "perception_obstacle"', content)

    # 创建一个空的DataFrame用于存储数据
    df = pd.DataFrame()
    
    for block in blocks:
        data = dict()
        ts_match = re.search(r'timestamp_sec: ([\d.+-]+)', block)
        if ts_match:
            ts = float(ts_match.group(1))   
            data['npc_ts'] = ts
        subblocks = re.split(r'perception_obstacle \{', block)
        # 计算并打印perception_obstacle的数量
        num_perception_obstacles = len(subblocks) - 1
        for i, subblock in enumerate(subblocks):
            # 使用正则表达式提取所需的数据
            id_match = re.search(r'id: (\d+)', subblock)
            x_match = re.search(r'position \{\s+x: ([\d.+-]+)', subblock)
            y_match = re.search(r'y: ([\d.+-]+)', subblock)
            theta_match = re.search(r'theta: ([\d.+-]+)', subblock)
            
            # 如果在块中找到了所有所需的数据，则将其添加到DataFrame中
            if id_match and x_match and y_match and theta_match:
                npc_id = int(id_match.group(1))
                x = float(x_match.group(1))
                y = float(y_match.group(1))
                theta = float(theta_match.group(1))

                # 动态创建列名
                data.update({
                    f'NPC{npc_id}X': x,
                    f'NPC{npc_id}Y': y,
                    f'NPC{npc_id}Theta': theta,
                })

        if data:
            # 使用pandas.concat将数据添加到DataFrame中
            df = pd.concat([df, pd.DataFrame([data])], ignore_index=True)
    
    # 使用timestamp_sec列对DataFrame进行排序（如果需要的话）
    # df = df.sort_values(by='timestamp_sec').reset_index(drop=True)
    
    return df


In [23]:
def ego_from_txt(filepath):
    f = open(filepath, 'r')
    fframe = f.readlines()
    
    #append timestamp
    df_ts = pd.DataFrame(columns=['ego_ts'])
    #append position
    df_lc = pd.DataFrame(columns=['egoX','egoY','heading'])
    
    for row in fframe:
        if row.find('timestamp_sec') != -1:
            ts = re.findall("(?<=[AZaz])?[0-9.+-]+",row) #exclude "timestamp_sec:"
            ts = float(ts[0]) #convert to data format
            l_no = fframe.index(row)
            
            lc_x = re.findall("(?<=[AZaz])?[0-9.+-]+",fframe[l_no + 6])
            lc_x = float(lc_x[0])
            lc_y = re.findall("(?<=[AZaz])?[0-9.+-]+",fframe[l_no + 7])
            lc_y = float(lc_y[0])
            heading = re.findall("(?<=[AZaz])?[0-9.+-]+",fframe[l_no +31])
            heading = float(heading[0])
            
            d_0 = {'ego_ts':[ts]}
            d_df0 = pd.DataFrame(data=d_0)
            df_ts = pd.concat([df_ts, d_df0])

            d_1 = {'egoX':[lc_x], 'egoY':[lc_y],'heading':[heading]}
            d_df1 = pd.DataFrame(data=d_1)
            df_lc = pd.concat([df_lc, d_df1])
    df = pd.concat([df_ts, df_lc], axis=1)

    return df

In [24]:
def cmd_from_txt(filepath):
    f = open(filepath, 'r')
    fframe = f.readlines()
    
    # Initialize an empty DataFrame
    df = pd.DataFrame(columns=['cmd_ts', 'throttle', 'brake', 'steering'])
    
    for l_no, row in enumerate(fframe):
        if re.match(r'^header {', row):
            ts = re.findall("(?<=[AZaz])?[0-9.+-]+", fframe[l_no + 1])
            ts = float(ts[0])

            throttle = re.findall("(?<=[AZaz])?[0-9.+-]+", fframe[l_no + 11])
            if throttle:
                throttle = float(throttle[0])
            else:
                throttle = re.findall("(?<=[AZaz])?[0-9.+-]+", fframe[l_no + 12])
                if throttle:
                    throttle = float(throttle[0])
                else:
                    print(f"Error in {filepath}, line {l_no}: Throttle value not found")
                    print(f"timestamp = {ts}")

            # throttle = float(throttle[0])

            brake = re.findall("(?<=[AZaz])?[0-9.+-]+", fframe[l_no + 12])
            brake = float(brake[0])

            steering = re.findall("(?<=[AZaz])?[0-9.+-]+", fframe[l_no + 14])
            steering = float(steering[0])

            col_df = pd.concat([pd.DataFrame({'cmd_ts': [ts]}), 
                                 pd.DataFrame({'throttle': [throttle]}), 
                                 pd.DataFrame({'brake': [brake]}), 
                                 pd.DataFrame({'steering': [steering]})], axis=1)
            
            df = pd.concat([df, col_df], ignore_index=True)

    return df

In [25]:
def align_and_average(serial):
    data_path = root_path
    df_npc = npc_from_txt_wid(data_path+f'/txt_record/{serial}_obstacle.txt')
    df_ego = ego_from_txt(data_path+f'/txt_record/{serial}_pose.txt')
    df_cmd = cmd_from_txt(data_path+f'/txt_record/{serial}_control.txt')
## 根据是否是obstacle来的选择要不要手动对齐数据
    delta_ts = df_ego['ego_ts'].iloc[0] - df_npc['npc_ts'].iloc[0]
    df_npc['npc_ts'] += delta_ts

    # 使用列表来收集有效的数据
    aligned_ego_list = []
    avg_cmd_list = []
    aligned_npc_list = []
    
    # 上一个npc时间戳的索引
    last_npc_idx = 0
    
    for i in range(df_npc.shape[0]):
        # 对齐ego的时间戳
        ego_indices = np.where(df_ego['ego_ts'] < df_npc['npc_ts'].iloc[i])[0]
        if ego_indices.size == 0:
            continue
        ego_idx = ego_indices[-1]
        
        # 平均cmd的数据
        cmd_indices = np.where((df_cmd['cmd_ts'] >= df_npc['npc_ts'].iloc[last_npc_idx]) & 
                               (df_cmd['cmd_ts'] < df_npc['npc_ts'].iloc[i]))[0]
        
        # 检查cmd_indices是否为空
        if cmd_indices.size > 0:
            aligned_ego_list.append(df_ego.iloc[ego_idx])
            avg_cmd_list.append(df_cmd.iloc[cmd_indices].mean())
            aligned_npc_list.append(df_npc.iloc[i])
        
        last_npc_idx = i

    # 将列表转换为DataFrame
    df_aligned_ego = pd.DataFrame(aligned_ego_list).reset_index(drop=True)
    df_avg_cmd = pd.DataFrame(avg_cmd_list).reset_index(drop=True)
    df_aligned_npc = pd.DataFrame(aligned_npc_list).reset_index(drop=True)

    return df_aligned_ego, df_aligned_npc, df_avg_cmd

In [26]:
# # 测试用的，单独提取一个run的record
# aligned_ego, aligned_npc, avg_cmd = align_and_average(46)

# # print (aligned_ego)
# # print (avg_cmd)
# # 将numpy数组转换为pandas DataFrame
# df_ego = pd.DataFrame(aligned_ego)  # 请根据你的数据调整列名
# df_cmd = pd.DataFrame(avg_cmd)     # 请根据你的数据调整列名
# df_npc = pd.DataFrame(aligned_npc)         # 请根据你的数据调整列名

# # 保存为CSV文件
# df_ego.to_csv('assets/data200/aligned_ego.csv', index=False)
# df_cmd.to_csv('assets/data200/avg_cmd.csv', index=False)
# df_npc.to_csv('assets/data200/aligned_npc.csv', index=False)

In [27]:
# 单独处理的46
def process_serials(serials):
    merge_df = pd.DataFrame()
    
    for i in range(serials):
        # if i == 46:
        #     continue
        print(f"Processing serial {i}")
        aligned_ego, aligned_npc, avg_cmd = align_and_average(i)
        # 创建一个临时的DataFrame来存储这一轮的数据
        tmp = aligned_ego['ego_ts'].to_frame(name='time')
        tmp['id'] = i + 1  # 添加新列
        tmp = pd.concat([tmp, aligned_ego, aligned_npc, avg_cmd], axis=1)
        
        # 在尝试删除列之前，检查它们是否存在
        columns_to_drop = ['ego_ts', 'cmd_ts', 'npc_ts']
        for col in columns_to_drop:
            if col not in tmp.columns:
                print(f"Warning: Column '{col}' not found in DataFrame at iteration {i}. Available columns: {tmp.columns}")
            else:
                tmp = tmp.drop(columns=col)
        # 将这一轮的数据添加到merge_df中
        merge_df = pd.concat([merge_df, tmp], ignore_index=True)
    return merge_df


In [28]:
# 这里是提取静态feature的部分
import pandas as pd
import re

# 定义一个函数来解析文件并返回一个DataFrame
def parse_simulation_data(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    # 初始化空的列表来存储数据
    data = []
    
    # 初始化一个空的字典来临时存储每次迭代的数据
    iteration_data = {}
    
    # 遍历文件的每一行
    for line in lines:
        # 使用正则表达式来解析数据
        if "Iteration" in line:
            # 如果字典不为空，则将其添加到数据列表中，并清空字典
            if iteration_data:
                data.append(iteration_data)
                iteration_data = {}
            iteration_data['id'] = int(re.search(r'\d+', line).group())
        elif "Weather" in line:
            weather_data = re.findall(r'(\w+)=([\d.]+)', line)
            for item in weather_data:
                iteration_data[item[0]] = float(item[1])
        elif "Time of day" in line:
            iteration_data['daytime'] = float(re.search(r'[\d.]+', line).group())
        elif "Collision" in line:
            match = re.search(r"Collision: (True|False)", line)
            if match:
                # 如果找到了匹配的字符串，获取True或False的值
                value = match.group(1)
                # 根据True或False的值进行转换，并保存在字典中
                iteration_data['result'] = 1 if value == "True" else 0
    
    # 确保最后一次迭代的数据也被添加到数据列表中
    if iteration_data:
        data.append(iteration_data)
    
    # 将数据列表转换为DataFrame
    df = pd.DataFrame(data)
    
    return df

# 提取结果等静态部分 使用函数解析文件并获取DataFrame
file_path = root_path+'simulation_data.txt'
df = parse_simulation_data(file_path)

# 显示DataFrame的前几行
# print(df)
df.to_csv(root_path+'features.csv')

In [29]:
### 此处点击运行以上所有代码即可！！！修改总共的序列个数！！
# print(process_serials(6))
df_merge = pd.DataFrame()
df_merge = process_serials(71)
df_merge = df_merge.reindex(columns=['id', 'time'] + [col for col in df_merge.columns if col not in ['id', 'time']])
df_merge.insert(0, "", range(1, len(df_merge) + 1))
df_merge.to_csv(root_path+'data_processed.csv',index=False)

Processing serial 0
Processing serial 1
Processing serial 2
Processing serial 3
Processing serial 4
Processing serial 5
Processing serial 6
Processing serial 7
Processing serial 8
Processing serial 9
Processing serial 10
Processing serial 11
Processing serial 12
Processing serial 13
Processing serial 14
Processing serial 15
Processing serial 16
Processing serial 17
Processing serial 18
Processing serial 19
Processing serial 20
Processing serial 21
Processing serial 22
Processing serial 23
Processing serial 24
Processing serial 25
Processing serial 26
Processing serial 27
Processing serial 28
Processing serial 29
Processing serial 30
Processing serial 31
Processing serial 32
Processing serial 33
Processing serial 34
Processing serial 35
Processing serial 36
Processing serial 37
Processing serial 38
Processing serial 39
Processing serial 40
Processing serial 41
Processing serial 42
Processing serial 43
Processing serial 44
Processing serial 45
Processing serial 46
Processing serial 47
Pr

In [40]:
### 如果是直道场景，

    
def compute_perpendicular_distance(x, y, line):
    x, y = float(x), float(y)  # Convert x and y to float
    min_distance = float('inf')
    sign = 1  # Positive for left, negative for right
    
    for i in range(len(line) - 1):
        x1, y1 = float(line[i]['x']), float(line[i]['y'])  # Convert string to float
        x2, y2 = float(line[i+1]['x']), float(line[i+1]['y'])  # Convert string to float
        
        v_length = math.sqrt((x2-x1)**2 + (y2-y1)**2)
        dot_product = ((x-x1)*(x2-x1) + (y-y1)*(y2-y1)) / v_length
        
        if dot_product < 0:
            distance = math.sqrt((x-x1)**2 + (y-y1)**2)
        elif dot_product > v_length:
            distance = math.sqrt((x-x2)**2 + (y-y2)**2)
        else:
            # Compute the cross product to determine the side
            cross_product = (x - x1) * (y2 - y1) - (y - y1) * (x2 - x1)
            if cross_product < 0:
                sign = -1
            distance = abs((y2-y1)*x - (x2-x1)*y + x2*y1 - y2*x1) / v_length
        
        # Update the minimum distance with the correct sign
        if distance < min_distance:
            min_distance = distance * sign
            
    return min_distance


def find_lane_and_distance(x, y, lanes):
    if x and y:
        x, y = float(x), float(y)  # Convert x and y to float
    else:
        return None, None
    min_distance = float('inf')
    closest_lane_id = None
    
    for lane_id, lane_data in lanes.items():
        central_curve = lane_data['central_curve']
        distance = compute_perpendicular_distance(x, y, central_curve)
        
        # 更新最近的车道和距离
        if abs(distance) < abs(min_distance):
            min_distance = distance
            closest_lane_id = lane_id
          
    # 如果最近的车道距离小于2m，则返回该车道和距离，否则返回None
    if abs(min_distance) < 2:
        lane_number = re.findall(r'\d+', closest_lane_id)
        return lane_number[0] if lane_number else closest_lane_id, min_distance
    else:
        return None, None
    
# # test
# with open('assets/lane_info.json', 'r') as f:
#      lanes = json.load(f)
# x, y = 592418.56176757812, 4134988.5171813965  # Example point
# lane_id, distance = find_lane_and_distance(x, y, lanes)

# if lane_id:
#     print(f"The point ({x}, {y}) is on lane {lane_id} with a distance of {distance} to the central curve.")
# else:
#     print(lane_id, distance)



df = pd.read_csv(root_path+'data_processed.csv')
# Load lanes data from JSON
with open('assets/lane_info.json', 'r') as f:
    lanes = json.load(f)
# 遍历dataframe的每一行
for index, row in df.iterrows():
    # 对ego进行计算
    lane_id, distance = find_lane_and_distance(row['egoX'], row['egoY'],lanes)
    df.at[index, 'egoLane'] = lane_id
    df.at[index, 'egoLaneOffset'] = distance
    
    # 对NPC1进行计算
    lane_id, distance = find_lane_and_distance(row['NPC1X'], row['NPC1Y'],lanes)
    df.at[index, 'NPC1Lane'] = lane_id
    df.at[index, 'NPC1LaneOffset'] = distance
    
    # 对NPC2进行计算
    lane_id, distance = find_lane_and_distance(row['NPC2X'], row['NPC2Y'],lanes)
    df.at[index, 'NPC2Lane'] = lane_id
    df.at[index, 'NPC2LaneOffset'] = distance

# 将更新后的dataframe保存回CSV文件
df.to_csv(root_path+'data_processed.csv', index=False)


The point (592418.5617675781, 4134988.5171813965) is on lane 260 with a distance of 0.0 to the central curve.


In [29]:
### 如果录制的时候cut不及时,强行处理一下截断后面的：
#读一下df.to_csv(root_path+'features.csv')如果对应result是0的，就把(root_path+'data_processed.csv',index=False)的time50之后的都不要了
import pandas as pd
root_path = 'assets/1failtoyield/'
# 读取 features.csv 文件
features_df = pd.read_csv(root_path + 'features.csv')

# 找到 result 列为 0 的所有行的 id
ids_to_remove = features_df[features_df['result'] == 0]['id']

# 读取 data_processed.csv 文件
data_processed_df = pd.read_csv(root_path + 'data_processed.csv')

# 创建一个空 DataFrame 用于存储修改后的数据
data_processed_modified_df = pd.DataFrame()

for id in data_processed_df['id'].unique():
    if id in ids_to_remove.values:
        # 如果 id 的 result 为 0，只保留前 50 行
        data_to_add = data_processed_df[data_processed_df['id'] == id].head(50)
    else:
        # 如果 id 的 result 为 1，保留所有行
        data_to_add = data_processed_df[data_processed_df['id'] == id]
    
    data_processed_modified_df = pd.concat([data_processed_modified_df, data_to_add])

# 删除 "Unnamed: 0" 列
if 'Unnamed: 0' in data_processed_modified_df.columns:
    data_processed_modified_df.drop('Unnamed: 0', axis=1, inplace=True)

# 重置索引，使其从 1 开始
data_processed_modified_df.index = range(1, len(data_processed_modified_df) + 1)

# 保存 DataFrame，索引列没有列名
data_processed_modified_df.to_csv(root_path + 'data_processed_modified.csv', index=True, index_label='')

In [14]:
"""以下discarded: numpy overview"""
# def process_serial(serial):
#     aligned_ego, aligned_npc, avg_cmd = align_and_average(serial)

#     # 生成具体numpy元素
#     data_dict = {
#         'timestamp': aligned_ego[:, 0],
#         'ego_x': aligned_ego[:, 1],
#         'ego_y': aligned_ego[:, 2],
#         'ego_theta': aligned_ego[:, 3],
#         'npc_x': aligned_npc[:, 1],
#         'npc_y': aligned_npc[:, 2],
#         'npc_theta': aligned_npc[:, 3],
#         'throttle': avg_cmd[:, 1],
#         'brake': avg_cmd[:, 2],
#         'steering': avg_cmd[:, 3]
#     }
#     ## 这里是生成overview用的numpy长度
#     # data_dict = {
#     #     'timestamp': len(aligned_ego[:, 0]),
#     #     'ego_x': len(aligned_ego[:, 1]),
#     #     'ego_y': len(aligned_ego[:, 2]),
#     #     'ego_theta': len(aligned_ego[:, 3]),
#     #     'npc_x': len(aligned_npc[:, 1]),
#     #     'npc_y': len(aligned_npc[:, 2]),
#     #     'npc_theta': len(aligned_npc[:, 3]),
#     #     'throttle': len(avg_cmd[:, 1]),
#     #     'brake': len(avg_cmd[:, 2]),
#     #     'steering': len(avg_cmd[:, 3])
#     # }
#     return data_dict

# # 确保目录存在
# output_dir = 'assets/numpy'

# df_big = pd.DataFrame()  # 假设这是你的大dataframe

# for serial in range(1, 22):
#     serial_data = process_serial(serial)
    
#     # 为每个numpy数组保存一个文件，并在serial_data中保存文件名作为占位符
#     for key, array in serial_data.items():
#         filename = os.path.join(output_dir, f'output_data_{serial}_{key}.txt').replace('\\', '/')
#         np.savetxt(filename, array)
#         serial_data[key] = filename  # 保存文件名作为占位符
        
#     # 将serial_data添加到大dataframe中
#     df_big = pd.concat([df_big, pd.DataFrame([serial_data])], ignore_index=True)

# df_big.to_csv('assets/output_overview_np.csv', index=False)


# # 下面的方法是把numpy数组转换成str存储在csv的！！效率比较低！
# # for serial in range(1, 22):
# #     serial_data = process_serial(serial)
# #     # 将serial_data添加到大dataframe中
# #     df_big = pd.concat([df_big, pd.DataFrame([serial_data])], ignore_index=True)

# # df_big.to_csv('output_overview.csv', index=False)

以上是record的预处理，结果在assets\output_data.csv

In [1]:
import json
import math
import numpy as np
import pandas as pd
import os
from utils.txt_np import placeholder_to_np, np_to_placeholder

def compute_perpendicular_distance(x, y, line):
    x, y = float(x), float(y)  # Convert x and y to float
    min_distance = float('inf')
    sign = 1  # Positive for left, negative for right
    
    for i in range(len(line) - 1):
        x1, y1 = float(line[i]['x']), float(line[i]['y'])  # Convert string to float
        x2, y2 = float(line[i+1]['x']), float(line[i+1]['y'])  # Convert string to float
        
        v_length = math.sqrt((x2-x1)**2 + (y2-y1)**2)
        dot_product = ((x-x1)*(x2-x1) + (y-y1)*(y2-y1)) / v_length
        
        if dot_product < 0:
            distance = math.sqrt((x-x1)**2 + (y-y1)**2)
        elif dot_product > v_length:
            distance = math.sqrt((x-x2)**2 + (y-y2)**2)
        else:
            # Compute the cross product to determine the side
            cross_product = (x - x1) * (y2 - y1) - (y - y1) * (x2 - x1)
            if cross_product < 0:
                sign = -1
            distance = abs((y2-y1)*x - (x2-x1)*y + x2*y1 - y2*x1) / v_length
        
        # Update the minimum distance with the correct sign
        if distance < min_distance:
            min_distance = distance * sign
            
    return min_distance


def find_lane_and_distance(x, y, lanes):
    x, y = float(x), float(y)  # Convert x and y to float
    min_distance = float('inf')
    closest_lane_id = None
    
    for lane_id, lane_data in lanes.items():
        central_curve = lane_data['central_curve']
        distance = compute_perpendicular_distance(x, y, central_curve)
        
        # 更新最近的车道和距离
        if abs(distance) < abs(min_distance):
            min_distance = distance
            closest_lane_id = lane_id
          
    # 如果最近的车道距离小于2m，则返回该车道和距离，否则返回None
    if abs(min_distance) < 2:
        return closest_lane_id, min_distance
    else:
        return None, None

# test
# x, y = 592960.0, 4135065.0  # Example point
# lane_id, distance = find_lane_and_distance(x, y, lanes)

# if lane_id:
#     print(f"The point ({x}, {y}) is on lane {lane_id} with a distance of {distance} to the central curve.")
# else:
#     print(f"The point ({x}, {y}) is not on any lane.")

# 接收一行的数据，返回当前一行的结果
def process_data(lanes, ego_x_array, ego_y_array, npc_x_array, npc_y_array):
    ego_lane_list, ego_offset_list = [], []
    npc_lane_list, npc_offset_list = [], []
        
    # 迭代每个numpy数组中的每个值
    for ego_x, ego_y, npc_x, npc_y in zip(ego_x_array, ego_y_array, npc_x_array, npc_y_array):
        ego_lane, ego_offset = find_lane_and_distance(ego_x, ego_y, lanes)
        npc_lane, npc_offset = find_lane_and_distance(npc_x, npc_y, lanes)
        
        ego_lane_list.append(ego_lane)
        ego_offset_list.append(ego_offset)
        npc_lane_list.append(npc_lane)
        npc_offset_list.append(npc_offset)
    
    return ego_lane_list, ego_offset_list, npc_lane_list, npc_offset_list


# Load lanes data from JSON
with open('assets/lane_info.json', 'r') as f:
    lanes = json.load(f)

# 从CSV中读取数据
data = pd.read_csv('assets/output_overview_np.csv')

# Convert placeholders to numpy arrays
keys = ["ego_x", "ego_y", "npc_x", "npc_y"]
data = placeholder_to_np(data, keys)
# print(f"data1 = {data}")

# 确保列存在
for col in ['ego_lane', 'ego_offset', 'npc_lane', 'npc_offset']:
    if col not in data.columns:
        data[col] = np.nan

output_dir = 'assets/numpy'
# 在循环中处理每个序列
for index, row in data.iterrows():
    ego_lane_series, ego_offset_series, npc_lane_series, npc_offset_series = process_data(
        lanes, 
        np.array(row['ego_x']),  # Ensure it's a numpy array
        np.array(row['ego_y']),  # Ensure it's a numpy array
        np.array(row['npc_x']),  # Ensure it's a numpy array
        np.array(row['npc_y']))  # Ensure it's a numpy array

    # 为每个序列创建一个文件并保存
    filenames = {}
    for key, series in zip(['ego_lane', 'ego_offset', 'npc_lane', 'npc_offset'], 
                        [ego_lane_series, ego_offset_series, npc_lane_series, npc_offset_series]):
        filename = os.path.join(output_dir, f'{key}_{index+1}.txt').replace('\\', '/')
        if isinstance(series[0], (int, float, np.number)):  # Check if the first element is a number
            np.savetxt(filename, series, fmt='%s')  # Save as numbers
        else:
            with open(filename, 'w') as f:
                for item in series:
                    f.write("%s\n" % item)  # Save as strings
        filenames[key] = filename

    # 在DataFrame中存储文件的名称
    for key, filename in filenames.items():
        data.at[index, key] = filename


# 保存更新后的dataframe
# data.to_csv('assets/output_lane_interaction.csv', index=False)

# 1. 读取assets\output_overview_np.csv到一个新的DataFrame
overview_df = pd.read_csv('assets/output_overview_np.csv')

# 2. 使用pd.concat将新列并入这个DataFrame
# 确保两个DataFrame的索引是对齐的
combined_df = pd.concat([overview_df, data[['ego_lane', 'ego_offset', 'npc_lane', 'npc_offset']]], axis=1)

# 3. 将结果保存回assets\output_overview_np.csv
combined_df.to_csv('assets/output_overview_np.csv', index=False)


In [3]:
import pandas as pd
import numpy as np
import re

df = pd.read_csv('assets/output_overview_np.csv')

new_df = pd.DataFrame()

for index, row in df.iterrows():
    row_df = pd.DataFrame()
    for column in df.columns:
        try:
            with open(row[column], 'r') as file:
                data_lines = file.readlines()
                # print(data_lines)
                elem = [float(line.strip()) for line in data_lines if re.match(r'^-?\d+(\.\d+)?(e[+-]?\d+)?$', line.strip())]              
                # print(elem)
                # 如果elem列表为空，引发一个异常
                if not elem:
                    raise ValueError(f"Cannot convert content of file {row[column]} to float")
                col_df = pd.DataFrame({column: elem})
            # print(col_df)
        except:
            # print("!!")
            with open(row[column], 'r') as file:
                data_lines = file.readlines()
                elem = [] 
                for line in data_lines:
                    match = re.search(r'lane_(\d+)', line)
                    # print(match)
                    # break
                    if match:
                        elem.append(int(match.group(1)))
                    else:
                        try:
                            # 尝试将其转换为浮点数
                            elem.append(float(line.strip()))
                        except ValueError:
                            elem.append('nan')
                            print(f"Cannot parse line: {line.strip()} in file: {row[column]}")
        col_df = pd.DataFrame({column: elem})
        # print(col_df)
        # break
        # 将col_df加到row_df
        if 'id' in row_df:
            row_df = pd.concat([row_df, col_df], axis=1)
        else:
            row_df = col_df
            row_df['id'] = [index+1] * len(elem)  # 初始化id列
    # 将row_df加到new_df
    new_df = pd.concat([new_df, row_df], axis=0)
    # print(new_df)

new_df.insert(0, 'sequence', range(1, 1 + len(new_df)))
# 保存新的数据集
new_df.to_csv('transformed_data.csv', index=False)


Cannot parse line: None in file: assets/numpy/ego_lane_1.txt
Cannot parse line: None in file: assets/numpy/ego_lane_1.txt
Cannot parse line: None in file: assets/numpy/npc_lane_1.txt
Cannot parse line: None in file: assets/numpy/ego_lane_2.txt
Cannot parse line: None in file: assets/numpy/ego_lane_3.txt
Cannot parse line: None in file: assets/numpy/npc_lane_3.txt
Cannot parse line: None in file: assets/numpy/ego_lane_4.txt
Cannot parse line: None in file: assets/numpy/ego_lane_5.txt
Cannot parse line: None in file: assets/numpy/ego_lane_6.txt
Cannot parse line: None in file: assets/numpy/ego_lane_6.txt
Cannot parse line: None in file: assets/numpy/ego_lane_7.txt
Cannot parse line: None in file: assets/numpy/ego_lane_7.txt
Cannot parse line: None in file: assets/numpy/ego_lane_9.txt
Cannot parse line: None in file: assets/numpy/ego_lane_11.txt
Cannot parse line: None in file: assets/numpy/ego_lane_14.txt
Cannot parse line: None in file: assets/numpy/ego_lane_15.txt
Cannot parse line: No